# Collaborative Filtering

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore recommendation systems based on Collaborative Filtering and implement simple version of one using Python and the Pandas library.


In [1]:
import pandas as pd
import numpy as np
from math import sqrt

In [7]:
movies_df = pd.read_csv('./movies.csv')
ratings_df = pd.read_csv('./ratings.csv')

In [8]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [9]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


So each movie has a unique ID, a title with its release year along with it (Which may contain unicode characters) and several different genres in the same field. Let's remove the year from the title column and place it into its own one by using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.extract.html?utm_medium=Exinfluencer\&utm_source=Exinfluencer\&utm_content=000026UJ\&utm_term=10006555\&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01#pandas.Series.str.extract) function that Pandas has.


Let's remove the year from the **title** column by using pandas' replace function and store in a new **year** column.


In [10]:
#new col 'year' to store year of release mentioned in title col
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))')
#Removing () from year column
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)')
#Replacing (year) in title col with ''
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Removing empty spaces from title col
movies_df.title = movies_df.title.apply(lambda x: x.strip())

<ipython-input-10-7d096d30b165>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')


In [11]:
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also drop the genres column since we won't need it for this particular recommendation system.


In [12]:
movies_df.drop(['genres'], axis=1, inplace=True)

In [13]:
movies_df.head()

,movieId,title,year
0,1,Toy Story,1995
1,2,Jumanji,1995
2,3,Grumpier Old Men,1995
3,4,Waiting to Exhale,1995
4,5,Father of the Bride Part II,1995


Every row in the ratings dataframe has a user id associated with at least one movie, a rating and a timestamp showing when they reviewed it. We won't be needing the timestamp column, so let's drop it to save on memory.


In [14]:
ratings_df.drop('timestamp', 1, inplace=True)

In [15]:
ratings_df.head()

,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


<hr>

<a id="ref3"></a>

# Collaborative Filtering

Now it's time to start our work on recommendation systems.

The first technique we're going to take a look at is called **Collaborative Filtering**, which is also known as **User-User Filtering**. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the **Pearson Correlation Function**.

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/images/User_Item.png" width=800px>

The process for creating a User Based recommendation system is as follows:

*   Select a user with the movies the user has watched
*   Based on his rating to movies, find the top X neighbours
*   Get the watched movie record of the user for each neighbour.
*   Calculate a similarity score using some formula
*   Recommend the items with the highest score

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .


In [16]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
        ]
inputMovies = pd.DataFrame(userInput)
inputMovies

,title,rating
0,"Breakfast Club, The",5.0
1,Toy Story,3.5
2,Jumanji,2.0
3,Pulp Fiction,5.0
4,Akira,4.5


In [22]:
#Add movieId to the above table
inputId = movies_df[movies_df['title'].isin(inputMovies.title)]

inputMovies = pd.merge(inputId, inputMovies)

inputMovies

,movieId,title,year,rating
0,1,Toy Story,1995,3.5
1,2,Jumanji,1995,2.0
2,296,Pulp Fiction,1994,5.0
3,1274,Akira,1988,4.5
4,1968,"Breakfast Club, The",1985,5.0


In [23]:
#Drop year col from inputMovies
inputMovies.drop(['year'], axis=1, inplace=True)
inputMovies.head()

,movieId,title,rating
0,1,Toy Story,3.5
1,2,Jumanji,2.0
2,296,Pulp Fiction,5.0
3,1274,Akira,4.5
4,1968,"Breakfast Club, The",5.0


#### The users who has seen the same movies

Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.


In [25]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = ratings_df[ratings_df.movieId.isin(inputMovies.movieId)]
userSubset.head()

,userId,movieId,rating
19,4,296,4.0
441,12,1968,3.0
479,13,2,2.0
531,13,1274,5.0
681,14,296,2.0


We now group up the rows by user ID.


In [27]:
userSubsetgroup = userSubset.groupby(['userId'])
userSubsetgroup.get_group(1130)

,userId,movieId,rating
104167,1130,1,0.5
104168,1130,2,4.0
104214,1130,296,4.0
104363,1130,1274,4.5
104443,1130,1968,4.5


Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.


In [34]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetgroup = sorted(userSubsetgroup, key=lambda x: len(x[1]), reverse=True)

In [35]:
userSubsetgroup[:3]

[(247307,
            userId  movieId  rating
  22842095  247307        1     4.0
  22842096  247307        2     4.5
  22842125  247307      296     2.0
  22842207  247307     1274     3.0
  22842243  247307     1968     5.0),
 (246999,
            userId  movieId  rating
  22817271  246999        1     3.0
  22817272  246999        2     3.0
  22817309  246999      296     4.0
  22817426  246999     1274     5.0
  22817492  246999     1968     5.0),
 (246959,
            userId  movieId  rating
  22813540  246959        1     5.0
  22813541  246959        2     3.0
  22813616  246959      296     5.0
  22813810  246959     1274     4.0
  22813935  246959     1968     4.0)]

#### Similarity of users to input user

Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.\
we're going to find out how similar each user is to the input through the **Pearson Correlation Coefficient**. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below.

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 \* Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation.

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.


In [36]:
userSubsetgroup = userSubsetgroup[:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient.


In [40]:
pcd = {}

for name, group in userSubsetgroup: #Name=groupby field, Group=the rest of the entries for a name
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
        #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
     #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
        #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pcd[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pcd[name] = 0

In [44]:
pcd

{247307: -0.4071639637402853,
 246999: 0.7844645405527362,
 246959: 0.5860090386731196,
 246533: 0.8770580193070306,
 246506: 0.6784622064861935,
 246487: 0.7674257668936502,
 246481: 0.07520710469952337,
 246393: 0.9607689228305227,
 246207: 0.8320502943378437,
 245719: 0.9315516419063742,
 245568: 0.2644429426739724,
 245483: 0.905206050195776,
 245423: 0.8362420100070908,
 245326: 0.18941568257056735,
 245274: -0.36569278269801575,
 245021: 0.9073229767805335,
 244740: 0.7240929464269849,
 244657: 0.8790135580096794,
 244636: 0.29417420270727607,
 244577: 0.358057437019716,
 244212: -0.2644429426739726,
 244176: 0.8156473902861853,
 243600: -0.7093169886164387,
 243271: 0.716114874039432,
 243182: 0.45124262819713973,
 242908: 0.49334513586020373,
 242882: 0.8770580193070306,
 242713: 0.8320502943378437,
 242672: 0.6577935144802716,
 242604: 0.8204126541423654,
 242582: 0.4902903378454601,
 242153: 0.11720180773462392,
 241887: 0.5097796189288658,
 241859: 0.8320502943378437,
 24166

In [46]:
pear_df = pd.DataFrame.from_dict(pcd, orient='index')
pear_df.columns = ['SimilarityIndex']
pear_df['userId'] = pear_df.index
pear_df.index = range(len(pear_df))
pear_df.head()

,SimilarityIndex,userId
0,-0.407164,247307
1,0.784465,246999
2,0.586009,246959
3,0.877058,246533
4,0.678462,246506


In [53]:
topUsers = pear_df.sort_values(by='SimilarityIndex',ascending=False)[:50]


Now, let's start recommending movies to the input user.

#### Rating of selected users to all movies

We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our **pearsonDF** from the ratings dataframe and then store their correlation in a new column called \_similarityIndex". This is achieved below by merging of these two tables.


In [54]:
topUsersRating=topUsers.merge(ratings_df, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

,SimilarityIndex,userId,movieId,rating
0,0.98669,232983,1,4.0
1,0.98669,232983,2,3.0
2,0.98669,232983,29,5.0
3,0.98669,232983,32,4.0
4,0.98669,232983,39,4.0


Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:


In [56]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['SimilarityIndex']*topUsersRating['rating']
topUsersRating.head()

,SimilarityIndex,userId,movieId,rating,weightedRating
0,0.98669,232983,1,4.0,3.946761
1,0.98669,232983,2,3.0,2.960071
2,0.98669,232983,29,5.0,4.933451
3,0.98669,232983,32,4.0,3.946761
4,0.98669,232983,39,4.0,3.946761


In [58]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['SimilarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

,sum_similarityIndex,sum_weightedRating
movieId,,
1,39.943771,153.662705
2,39.943771,106.878566
3,13.719193,34.276935
4,0.716115,2.148345
5,14.132802,36.580964


In [59]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

,weighted average recommendation score,movieId
movieId,,
1,3.846975,1
2,2.675725,2
3,2.498466,3
4,3.000000,4
5,2.588373,5


In [60]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

,weighted average recommendation score,movieId
movieId,,
80693,5.0,80693
1181,5.0,1181
3491,5.0,3491
26151,5.0,26151
116,5.0,116
7939,5.0,7939
117109,5.0,117109
1150,5.0,1150
3415,5.0,3415
